In [83]:
import os
import pandas as pd
import numpy as np

In [84]:
# ot paths

# data are rooted in github\\nkelites\\data
# ogree path from gui\\ to data\\
abs_path = os.path.abspath(os.getcwd())
ot_path = abs_path + '\\..\\..\\data\\combined data\\combined data - 1 cleaning\\cleaning step 2 - career\\2.8 orgtree with gui\\'


In [85]:
# ot filename

ot_fname = '1.1 orgtree_add fields.xlsx'

In [86]:
# orgtree dataframe
print(abs_path)
print(ot_path)
print(ot_fname)
ot = pd.read_excel(ot_path+ot_fname,dtype=str)
ot.shape

C:\Users\seoul\Dropbox\00 technical\github\nkelites\gui\1 orgtree gui
C:\Users\seoul\Dropbox\00 technical\github\nkelites\gui\1 orgtree gui\..\..\data\combined data\combined data - 1 cleaning\cleaning step 2 - career\2.8 orgtree with gui\
1.1 orgtree_add fields.xlsx


(2368, 19)

In [87]:
ot.columns

Index(['InstitutionType', 'OrgType', 'PrimaryInstitution', 'OrgName',
       'PI_Index', 'OrgRank', 'P1', 'P2', 'P3', 'Alias_OrgName',
       'LinkToNext_PI', 'LinkToNext_Org', 'LinkToNext_Year', 'Notes',
       'L1_Index', 'L2_Index', 'L3_Index', 'L4_Index', 'L5_Index'],
      dtype='object')

In [88]:
ot.head(2)

,InstitutionType,OrgType,PrimaryInstitution,OrgName,PI_Index,OrgRank,P1,P2,P3,Alias_OrgName,LinkToNext_PI,LinkToNext_Org,LinkToNext_Year,Notes,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index
0,노동당,NaN,노동당,NaN,1.0.0.0.0.0,0,"위원장,책임비서,총비서",제1비서,NaN,NaN,Current,Current,Current,NaN,1,0,0,0,0
1,당외곽및사회단체_경제부문(별책),NaN,신포원양수산연합기업소,NaN,1.0.0.0.0.0,0,"지배인,당책임비서","기사장,당비서",NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0,0


In [89]:
# GenerateParentKey

def GenerateParentKey(p,o):
    
    # OrgName is null
    if o is np.nan:
        pk = np.nan
    
    # OrgName is not composite. Then PrimaryInstitution is Parent_Key
    elif "_" not in o:
        pk = p
        
    # OrgName is composite. Then PrimaryInstitution + OrgName - the lowest child 
    else:
        pk = p+"_"+o
        pk = "_".join(pk.split("_")[0:-1])
        
    return pk

In [90]:
# GenerateChildKey

def GenerateChildKey(p,o):
    
    # OrgName is null
    if o is np.nan:
        ck = p
    
    # OrgName is not null. Then Org_Key is PrimaryInstitution + OrgName
    else:
        ck = p + "_" + o
        
    return ck

In [91]:
# GenerateChildName

def GenerateChildName(p,o):
    
    # OrgName is null. Then ChildName is PrimaryInstitution
    if o is np.nan:
        cn = p
    
    # OrgName is not composite. Then ChildName is OrgName
    elif "_" not in o:
        cn = o
        
    # OrgName is composite. Then ChildName is the lowest child of OrgName
    else:
        cn = o.split("_")[-1]
        
    return cn

In [92]:
test = "xaeh_h"
"_" in test

True

In [93]:
ot["Parent_Key"]=ot.apply(lambda x: GenerateParentKey(x["PrimaryInstitution"],x["OrgName"]),axis=1)

In [94]:
ot["Child_Key"]=ot.apply(lambda x: GenerateChildKey(x["PrimaryInstitution"],x["OrgName"]),axis=1)

In [95]:
ot["Child_Name"]=ot.apply(lambda x: GenerateChildName(x["PrimaryInstitution"],x["OrgName"]),axis=1)

In [96]:
ot.head(5)

,InstitutionType,OrgType,PrimaryInstitution,OrgName,PI_Index,OrgRank,P1,P2,P3,Alias_OrgName,...,LinkToNext_Year,Notes,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index,Parent_Key,Child_Key,Child_Name
0,노동당,NaN,노동당,NaN,1.0.0.0.0.0,0,"위원장,책임비서,총비서",제1비서,NaN,NaN,...,Current,NaN,1,0,0,0,0,NaN,노동당,노동당
1,당외곽및사회단체_경제부문(별책),NaN,신포원양수산연합기업소,NaN,1.0.0.0.0.0,0,"지배인,당책임비서","기사장,당비서",NaN,NaN,...,NaN,NaN,1,0,0,0,0,NaN,신포원양수산연합기업소,신포원양수산연합기업소
2,당외곽및사회단체_경제부문(별책),NaN,태천3호발전소,NaN,1.0.0.0.0.0,0,소장,부소장,NaN,NaN,...,NaN,NaN,1,0,0,0,0,NaN,태천3호발전소,태천3호발전소
3,당외곽및사회단체_경제부문(별책),NaN,평양곡산공장,NaN,1.0.0.0.0.0,0,"지배인,당책임비서","기사장,당비서",부기사장,NaN,...,NaN,NaN,1,0,0,0,0,NaN,평양곡산공장,평양곡산공장
4,당외곽및사회단체_경제부문(별책),NaN,강계고려약공장,NaN,1.0.0.0.0.0,0,지배인,NaN,NaN,NaN,...,NaN,NaN,1,0,0,0,0,NaN,강계고려약공장,강계고려약공장


In [97]:
# OrgName is composite

ot_nonnull = ot[~ot.OrgName.isna()]
ot_nonnull[ot_nonnull.OrgName.str.contains("_")].head(5)

,InstitutionType,OrgType,PrimaryInstitution,OrgName,PI_Index,OrgRank,P1,P2,P3,Alias_OrgName,...,LinkToNext_Year,Notes,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index,Parent_Key,Child_Key,Child_Name
41,노동당,NaN,노동당,당중앙위원회_정치국,1.1.1.0.0.0,2,"위원장,상무위원,비서",위원,"후보위원,정의원",NaN,...,Current,NaN,1,1,1,0,0,노동당_당중앙위원회,노동당_당중앙위원회_정치국,정치국
42,노동당,NaN,노동당,당중앙위원회_군수공업부,1.1.10.0.0.0,2,부장,"부부장,제1부부장",NaN,NaN,...,Current,NaN,1,1,10,0,0,노동당_당중앙위원회,노동당_당중앙위원회_군수공업부,군수공업부
43,노동당,NaN,노동당,당중앙위원회_군수공업부_제2경제위원회,1.1.10.201.0.0,3,위원장,NaN,NaN,NaN,...,Current,NaN,1,1,10,201,0,노동당_당중앙위원회_군수공업부,노동당_당중앙위원회_군수공업부_제2경제위원회,제2경제위원회
44,노동당,NaN,노동당,당중앙위원회_군정지도부,1.1.11.0.0.0,2,부장,NaN,NaN,NaN,...,Current,NaN,1,1,11,0,0,노동당_당중앙위원회,노동당_당중앙위원회_군정지도부,군정지도부
45,노동당,NaN,노동당,당중앙위원회_규율조사부,1.1.12.0.0.0,2,부장,NaN,NaN,NaN,...,Current,NaN,1,1,12,0,0,노동당_당중앙위원회,노동당_당중앙위원회_규율조사부,규율조사부


In [98]:
ot_nonnull[~ot_nonnull.OrgName.str.contains("_")].head(5)

,InstitutionType,OrgType,PrimaryInstitution,OrgName,PI_Index,OrgRank,P1,P2,P3,Alias_OrgName,...,LinkToNext_Year,Notes,L1_Index,L2_Index,L3_Index,L4_Index,L5_Index,Parent_Key,Child_Key,Child_Name
28,노동당,NaN,노동당,당중앙위원회,1.1.0.0.0.0,1,총비서,NaN,"과장,지도원,후보위원,위원,책임지도원,고문",NaN,...,Current,NaN,1,1,0,0,0,노동당,노동당_당중앙위원회,당중앙위원회
29,당외곽및사회단체_사회부문(별책),NaN,국제문제연구소,제1연구소,1.1.0.0.0.0,1,소장,부소장,NaN,NaN,...,Current,NaN,1,1,0,0,0,국제문제연구소,국제문제연구소_제1연구소,제1연구소
30,당외곽및사회단체_사회부문(별책),NaN,국제문제연구소,제2연구소,1.1.0.0.0.0,1,소장,부소장,NaN,NaN,...,Current,NaN,1,1,0,0,0,국제문제연구소,국제문제연구소_제2연구소,제2연구소
31,당외곽및사회단체_사회부문(별책),NaN,국제문제연구소,환경연구소,1.1.0.0.0.0,1,소장,부소장,NaN,NaN,...,Current,NaN,1,1,0,0,0,국제문제연구소,국제문제연구소_환경연구소,환경연구소
32,당외곽및사회단체_사회부문(별책),NaN,금성청년출판사,`,1.1.0.0.0.0,1,부장,부부장,논설원,NaN,...,NaN,NaN,1,1,0,0,0,금성청년출판사,금성청년출판사_`,`


In [99]:
# save over ot_fname = '1.1 orgtree_add fields.xlsx'
ot.to_excel(ot_path+ot_fname,index=False)